In [40]:
# import bokeh
from bokeh.models import DatetimeTickFormatter
from bokeh.io import output_notebook, show, save
from bokeh.plotting import figure
import numpy as np
import pandas as pd
import sqlite3


def db_connect():
    conn = sqlite3.connect('output/sales_new_seed.db')
    return conn


In [41]:
year_query = "SELECT * FROM sales AS s WHERE strftime('%Y-%m', s.sale_date) = '2013-01'"

base_query = "SELECT * FROM sales LIMIT 100"

batch_month_totals = "SELECT sale_date, SUM(sale_value) AS sale_value FROM sales WHERE strftime('%Y-%m', sale_date) = '2014-01' GROUP BY sale_date;"

sql_frame = pd.read_sql_query(batch_month_totals, db_connect(), parse_dates=['sale_date'])

values = [(x/100)/1000 for x in sql_frame['sale_value'].tolist()]

dates = [ts.date() for ts in sql_frame['sale_date'].tolist()]

sql_frame.sort_values(by=['sale_date']).head()


,sale_date,sale_value
0,2014-01-02,88175174
1,2014-01-06,172128247
2,2014-01-07,97039882
3,2014-01-08,146100437
4,2014-01-09,102932675


In [42]:
output_notebook()

plot = figure(plot_width=600, plot_height=600, tools='pan,box_zoom,reset')

plot.vbar(x=dates, width=0.5, bottom=0, top=values)
plot.xaxis.formatter=DatetimeTickFormatter(days=['%m/%d/%Y'])

show(plot)

Loading BokehJS ...